In [1]:
import os, sys, re, pickle, torch
import numpy as np
from numpy.random import default_rng

import torch
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import time
import sys, os
import pickle

/home/admin/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
kg_path = 'datasets/www_data/www_data/AmazonBook/kg/train.dat'
rec_path = 'datasets/www_data/www_data/AmazonBook/rs/ratings.txt'

kg = np.genfromtxt(kg_path, delimiter='\t', dtype=np.int32)
rec = np.genfromtxt(rec_path, delimiter='\t', dtype=None)


/tmp/ipykernel_526261/2099356923.py:5: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  rec = np.genfromtxt(rec_path, delimiter='\t', dtype=None)


In [57]:
str(rec[0][0])

"b'A18J1IH6920OXC'"

In [81]:
rec_users = []
rec_items = []
for i in range(rec.shape[0]):
    if int(rec[i][2]) >= 4:
        rec_users.append(re.search('(?<=\')(.*?)(?=\')', str(rec[i][0])).group(0))
        rec_items.append(re.sub("^0+", "", re.search('(?<=\')(.*?)(?=\')', str(rec[i][1])).group(0)))

In [50]:

kg[:,1] += 1 # offset
TOTAL_FB_IDS = np.max(kg) # total number of default kg pairs (# rel << # entities)

In [77]:

item2kg_path = 'datasets/www_data/www_data/AmazonBook/rs/i2kg_map.tsv'
emap_path = 'datasets/www_data/www_data/AmazonBook/rs/e_map.dat'
ml2fb_map = {}
with open(item2kg_path) as f:
    for line in f:
        ml_id = re.search('(.+?)\t', line)
        fb_http = re.search('\t(.+?)\n', line)
        ml2fb_map.update({re.sub("^0+", "", ml_id.group(1)) : fb_http.group(1)})


In [87]:
# maps free base html links to free base id's (final format)
id2html_map = {}
fb2id_map = {}
with open(emap_path) as f:
    for kg_id, line in enumerate(f):
        fb_http = re.search('\t(.+?)\n', line)
        fb2id_map.update({fb_http.group(1) : kg_id})
        id2html_map.update({kg_id : fb_http.group(1)})

In [ ]:
rec_users_kept = []
rec_items_converted = []
# convert movielens id's to freebase id's
i = 0
j = 0
while True:
    if i == len(rec_items):
        break
    if rec_items[i] in ml2fb_map: 
        # get correct freebase id from data
        fb_http = ml2fb_map[rec_items[i]]
        fb_id = fb2id_map[fb_http]
        rec_items_converted.append(fb_id)
        rec_users_kept.append(rec_users[i])
        i += 1

    j += 1
    print("1",j)